## Test deepSDF

In [2]:
import os
import sys
# add your module path
module_path = os.path.abspath(os.path.join('..')) # '.'
if module_path not in sys.path:
    sys.path.append(module_path)
# change your base path
os.chdir('../') # '../'
print(os.getcwd())

/home/yyjau/Documents/object_code/DeepSDF


In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import deep_sdf
from networks.deep_sdf_decoder import Decoder

## Test networks

In [151]:
latent_size = 256
dims = [ 512, 512, 512, 512, 512, 512, 512, 512 ]
net = Decoder(latent_size, dims)

# test decoder
experiment_directory = 'examples/chairs'
def test_net(experiment_directory):
#     experiment_directory = 'examples/chairs'
    specs = ws.load_experiment_specifications(experiment_directory)
#     latent_size = specs["CodeLength"]
    latent_size = 0
    
    net = Decoder(latent_size, **specs["NetworkSpecs"])
    return net

net = test_net(experiment_directory)

In [164]:
from networks.igr_decoder import Decoder
experiment_directory = 'examples/chairs_igr/'
def test_net_igr(experiment_directory):
#     experiment_directory = 'examples/chairs'
    specs = ws.load_experiment_specifications(experiment_directory)
#     latent_size = specs["CodeLength"]
    latent_size = 0
    
    net = Decoder(3+latent_size, **specs["NetworkSpecs"])
    return net

net = test_net_igr(experiment_directory)

In [165]:
print(net)

Decoder(
  (lin0): Linear(in_features=3, out_features=512, bias=True)
  (lin1): Linear(in_features=512, out_features=512, bias=True)
  (lin2): Linear(in_features=512, out_features=512, bias=True)
  (lin3): Linear(in_features=512, out_features=509, bias=True)
  (lin4): Linear(in_features=512, out_features=512, bias=True)
  (lin5): Linear(in_features=512, out_features=512, bias=True)
  (lin6): Linear(in_features=512, out_features=512, bias=True)
  (lin7): Linear(in_features=512, out_features=512, bias=True)
  (lin8): Linear(in_features=512, out_features=1, bias=True)
  (activation): Softplus(beta=100, threshold=20)
)


In [166]:
from torchsummary import summary
import torch
channels = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = net.to(device)
summary(net, input_size=(1,channels))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 512]           2,048
          Softplus-2               [-1, 1, 512]               0
            Linear-3               [-1, 1, 512]         262,656
          Softplus-4               [-1, 1, 512]               0
            Linear-5               [-1, 1, 512]         262,656
          Softplus-6               [-1, 1, 512]               0
            Linear-7               [-1, 1, 509]         261,117
          Softplus-8               [-1, 1, 509]               0
            Linear-9               [-1, 1, 512]         262,656
         Softplus-10               [-1, 1, 512]               0
           Linear-11               [-1, 1, 512]         262,656
         Softplus-12               [-1, 1, 512]               0
           Linear-13               [-1, 1, 512]         262,656
         Softplus-14               [-1,

In [149]:
from IGR.model.network import ImplicitNet
d_in = 3
latent_size = 0
dims = [ 512, 512, 512, 512, 512, 512, 512, 512 ]
net_igr = ImplicitNet(d_in+latent_size, dims, skip_in = [4])
print(net_igr)


ImplicitNet(
  (lin0): Linear(in_features=3, out_features=512, bias=True)
  (lin1): Linear(in_features=512, out_features=512, bias=True)
  (lin2): Linear(in_features=512, out_features=512, bias=True)
  (lin3): Linear(in_features=512, out_features=509, bias=True)
  (lin4): Linear(in_features=512, out_features=512, bias=True)
  (lin5): Linear(in_features=512, out_features=512, bias=True)
  (lin6): Linear(in_features=512, out_features=512, bias=True)
  (lin7): Linear(in_features=512, out_features=512, bias=True)
  (lin8): Linear(in_features=512, out_features=1, bias=True)
  (activation): Softplus(beta=100, threshold=20)
)


In [150]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net_igr = net_igr.to(device)
summary(net_igr, input_size=(1,d_in+latent_size))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 512]           2,048
          Softplus-2               [-1, 1, 512]               0
            Linear-3               [-1, 1, 512]         262,656
          Softplus-4               [-1, 1, 512]               0
            Linear-5               [-1, 1, 512]         262,656
          Softplus-6               [-1, 1, 512]               0
            Linear-7               [-1, 1, 509]         261,117
          Softplus-8               [-1, 1, 509]               0
            Linear-9               [-1, 1, 512]         262,656
         Softplus-10               [-1, 1, 512]               0
           Linear-11               [-1, 1, 512]         262,656
         Softplus-12               [-1, 1, 512]               0
           Linear-13               [-1, 1, 512]         262,656
         Softplus-14               [-1,

## Dataloader

In [6]:
import json
import deep_sdf.workspace as ws


experiment_directory = 'examples/chairs'
def test_dataset(experiment_directory):
    specs = ws.load_experiment_specifications(experiment_directory)
    
    data_source = specs["DataSource"]
    train_split_file = specs["TrainSplit"]
    num_samp_per_scene = specs["SamplesPerScene"]
    
    with open(train_split_file, "r") as f:
        train_split = json.load(f)
    
    sdf_dataset = deep_sdf.data.SDFSamples(
        data_source, train_split, num_samp_per_scene, load_ram=False
    )
    return sdf_dataset
    
sdf_dataset = test_dataset(experiment_directory)

In [7]:
sdf_dataset[0][0].shape
sdf_dataset[0][1]

0

In [32]:
def test_dataset_v2():
    experiment_directory = 'examples/chairs_igr/'
    specs = ws.load_experiment_specifications(experiment_directory)
    
    data_source = specs["DataSource"]
    print(data_source)
    train_split_file = specs["TrainSplit"]
    num_samp_per_scene = specs["SamplesPerScene"]
    
    with open(train_split_file, "r") as f:
        train_split = json.load(f)
    
    sdf_dataset = deep_sdf.data.SDFSurface(
        data_source, train_split, num_samp_per_scene, load_ram=False
    )
    return sdf_dataset
    
sdf_dataset = test_dataset_v2()

data
self.npyfiles: ShapeNetV2/03001627/1007e20d5e811b308351982a6e40cf41.ply


In [29]:
# data_source
idx = 2
pts = sdf_dataset[idx][0][0]
print(f"surface pts shape: {pts.shape}, mean: {pts.min(axis=0)}")
pts = sdf_dataset[idx][0][1]
print(f"sampled pts shape: {pts.shape}, mean: {pts.min(axis=0)}")

surface pts shape: torch.Size([8192, 3]), mean: torch.return_types.min(
values=tensor([-0.2113, -0.2997, -0.2393]),
indices=tensor([3137, 3051, 1326]))
sampled pts shape: torch.Size([8192, 3]), mean: torch.return_types.min(
values=tensor([-0.2238, -0.3196, -0.2596]),
indices=tensor([ 291, 4045, 7766]))


In [8]:
import trimesh
filename = 'data/SurfaceSamples/ShapeNetV2/03001627/1007e20d5e811b308351982a6e40cf41.ply'
npz = trimesh.load(filename)
npz

<trimesh.Trimesh(vertices.shape=(30000, 3), faces.shape=(10000, 3))>

In [9]:
import numpy as np
a = np.array(npz.vertices)
a.mean(axis=0)

array([-0.00944013, -0.08907329, -0.07379352])

In [10]:
import open3d as o3d
pcd = o3d.io.read_point_cloud(filename)
b = np.asarray(pcd.points)

In [11]:
b.mean(axis=0)

array([-0.00944013, -0.08907329, -0.07379352])

### Plot out samples

In [10]:
import numpy as np
import open3d as o3d
from open3d import JVisualizer


def open3d_vis(points, colors=None):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    if colors is not None:
        pcd.colors = o3d.utility.Vector3dVector(colors)

    visualizer = JVisualizer()
    visualizer.add_geometry(pcd)
    visualizer.show()

points = (np.random.rand(1000, 3) - 0.5) / 4
colors = np.random.rand(1000, 3)

open3d_vis(points, colors)

JVisualizer with 1 geometries

In [33]:
pcd = o3d.geometry.PointCloud()
idx = 3
mnfld_pnts, nonmnfld_pnts = sdf_dataset[idx][0][0], sdf_dataset[idx][0][1]

pcd.points = o3d.utility.Vector3dVector(nonmnfld_pnts)

pcd_2 = o3d.geometry.PointCloud()
pcd_2.points = o3d.utility.Vector3dVector(mnfld_pnts)

from notebooks.vis_tools import open3d_pcd_vis
# path_temp = '/home/yyjau/Documents/object_code/DeepSDF/examples/chairs_igr/Reconstructions/100/Meshes/ShapeNetV2/03001627/cc8fe2000b1471b2a85f7c85e000fc79.ply'
# pcd = o3d.io.read_point_cloud(path_temp)
open3d_pcd_vis(pcd+pcd_2)

JVisualizer with 1 geometries

## Loss functions

In [130]:
# input, network
sdf_data, indices = sdf_dataset[0]
net_igr

ImplicitNet(
  (lin0): Linear(in_features=3, out_features=512, bias=True)
  (lin1): Linear(in_features=512, out_features=512, bias=True)
  (lin2): Linear(in_features=512, out_features=512, bias=True)
  (lin3): Linear(in_features=512, out_features=509, bias=True)
  (lin4): Linear(in_features=512, out_features=512, bias=True)
  (lin5): Linear(in_features=512, out_features=512, bias=True)
  (lin6): Linear(in_features=512, out_features=512, bias=True)
  (lin7): Linear(in_features=512, out_features=512, bias=True)
  (lin8): Linear(in_features=512, out_features=1, bias=True)
  (activation): Softplus(beta=100, threshold=20)
)

In [131]:
mnfld_pnts = mnfld_pnts.float()
mnfld_pnts.dtype

torch.float32

In [136]:
(mnfld_pnts, nonmnfld_pnts) = sdf_data
mnfld_pnts, nonmnfld_pnts = mnfld_pnts.to(device), nonmnfld_pnts.to(device)
mnfld_pnts.requires_grad_()
nonmnfld_pnts.requires_grad_()

# forward pass
mnfld_pred = net_igr(mnfld_pnts)
nonmnfld_pred = net_igr(nonmnfld_pnts)

In [137]:
pred.shape

from IGR.model.network import gradient
mnfld_grad = gradient(mnfld_pnts, mnfld_pred)
nonmnfld_grad = gradient(nonmnfld_pnts, nonmnfld_pred)


In [102]:
# from IGR.model.sample import NormalPerPoint
# global_sigma = 1.8
# local_sigma = 0.01
# sampler = NormalPerPoint(global_sigma, local_sigma)
# mnfld_pnts = mnfld_pnts.unsqueeze(0)
# nonmnfld_pnts = sampler.get_points(mnfld_pnts)

In [139]:
nonmnfld_pnts.shape
# mnfld_pnts.shape

torch.Size([8192, 3])

In [140]:
grad_lambda = 0.1
# manifold loss

mnfld_loss = (mnfld_pred.abs()).mean()

# eikonal loss

grad_loss = ((nonmnfld_grad.norm(2, dim=-1) - 1) ** 2).mean()

loss = mnfld_loss + grad_lambda * grad_loss

In [141]:
loss

tensor(0.4962, device='cuda:0', grad_fn=<AddBackward0>)